In [31]:
import pandas as pd
import numpy as np
import sys
import os

pd.set_option('display.max_columns', 100)
# Define our base directory for the analysis
os.chdir('/home/jovyan/RepTract/CELLPHONEDB/')

In [32]:
print(sys.version)

3.8.16 | packaged by conda-forge | (default, Feb  1 2023, 16:01:55) 
[GCC 11.3.0]


In [33]:
cpdb_file_path = '/nfs/team292/vl6/FetalReproductiveTract/v5.0.0/cellphonedb.zip' # this is the downloaded database
meta_file_path = '/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/input/meta.tsv'
counts_file_path = '/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/input/counts_normalised.h5ad'
microenvs_file_path = '/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/input/microenvironments.tsv'
degs_file_path = '/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/input/DEGs_upregulated_genes.tsv'
out_path = '/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/output/'

In [34]:
metadata = pd.read_csv(meta_file_path, sep = '\t')
metadata.head(3)

,Cell,cell_type
0,HD_F_GON14449733_ATTGGACTCCTTGACC,Urethral Plate_female
1,HD_F_GON14449733_GCGCAGTGTCTGGAGA,Urethral Plate_female
2,HD_F_GON14449733_CGCTATCCAGGAATGC,Corpus Spongiosum_female


In [35]:
import anndata

adata = anndata.read_h5ad(counts_file_path)
adata.shape

(4500, 21123)

In [36]:
list(adata.obs.index).sort() == list(metadata['Cell']).sort()

True

In [37]:
degs = pd.read_csv(degs_file_path, sep = '\t')
degs.head(3)

,cluster,gene,p_val_adj,p_val,avg_log2FC,pct.1,pct.2
0,Corpus Spongiosum_female,XIST,1.403236e-243,7.117604e-248,0.877075,0.617,0.009
1,Corpus Spongiosum_female,LUM,3.109563e-131,1.577257e-135,0.548699,0.930,0.701
2,Corpus Spongiosum_female,GAP43,3.670967e-117,1.862017e-121,0.570242,0.844,0.544


In [38]:
microenv = pd.read_csv(microenvs_file_path, sep = '\t')
microenv

,celltype,microenvironment
0,Corpus Spongiosum_male,Male
1,Urethral Plate_male,Male
2,Corpus Spongiosum_female,Female
3,Urethral Plate_female,Female


In [39]:
microenv.groupby('microenvironment')['celltype'].apply(lambda x : list(x.value_counts().index))

microenvironment
Female    [Corpus Spongiosum_female, Urethral Plate_female]
Male          [Corpus Spongiosum_male, Urethral Plate_male]
Name: celltype, dtype: object

In [40]:

# all cells in microenv are in meta
[ item in set(metadata['cell_type']) for item in set(microenv['celltype']) ]

[True, True, True, True]

In [41]:
# all cells in microenv are in meta - who is not?
list(set(microenv['celltype']) - set(metadata['cell_type']) )

[]

In [42]:

# all cells in degs are in meta
[ item in set(metadata['cell_type']) for item in set(degs['cluster']) ]


[True, True, True, True]

In [43]:
# all cells in degs are in meta - who is not?
list(set(degs['cluster']) - set(metadata['cell_type']) )

[]

In [44]:
from cellphonedb.src.core.methods import cpdb_degs_analysis_method
res = \
    cpdb_degs_analysis_method.call(
        cpdb_file_path = cpdb_file_path, 
        meta_file_path = meta_file_path, 
        counts_file_path = counts_file_path,
        degs_file_path = degs_file_path,
        counts_data = 'hgnc_symbol',
        microenvs_file_path=microenvs_file_path,
        threshold = 0.1,
        result_precision = 3,
        separator = '|',
        debug = False,
        output_path = out_path,
        output_suffix = None,
        score_interactions = True,
        threads = 4)

[ ][CORE][20/08/24-15:00:35][INFO] [Cluster DEGs Analysis] Threshold:0.1 Precision:3
Reading user files...
The following user files were loaded successfully:
/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/input/counts_normalised.h5ad
/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/input/meta.tsv
/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/input/microenvironments.tsv
/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/input/DEGs_upregulated_genes.tsv
[ ][CORE][20/08/24-15:00:39][INFO] Running Real Analysis
[ ][CORE][20/08/24-15:00:39][INFO] Limiting cluster combinations using microenvironments
[ ][CORE][20/08/24-15:00:39][INFO] Running DEGs-based Analysis
[ ][CORE][20/08/24-15:00:39][INFO] Building results
[ ][CORE][20/08/24-15:00:39][INFO] Scoring interactions: Filtering genes per cell type..


100%|██████████| 4/4 [00:00<00:00, 70.14it/s]

[ ][CORE][20/08/24-15:00:40][INFO] Scoring interactions: Calculating mean expression of each gene per group/cell type..



100%|██████████| 4/4 [00:00<00:00, 221.70it/s]


[ ][CORE][20/08/24-15:00:40][INFO] Scoring interactions: Calculating scores for all interactions and cell types..


100%|██████████| 8/8 [00:01<00:00,  5.90it/s]


Saved deconvoluted_result to /nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/output/degs_analysis_deconvoluted_result_08_20_2024_150042.txt
Saved deconvoluted_percents to /nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/output/degs_analysis_deconvoluted_percents_08_20_2024_150042.txt
Saved means_result to /nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/output/degs_analysis_means_result_08_20_2024_150042.txt
Saved relevant_interactions_result to /nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/output/degs_analysis_relevant_interactions_result_08_20_2024_150042.txt
Saved significant_means to /nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/output/degs_analysis_significant_means_08_20_2024_150042.txt
Saved interaction_scores to /nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/output/degs_analysis_interaction_scores_08_20_2024_150042.txt


In [45]:
res.keys()

dict_keys(['deconvoluted_result', 'deconvoluted_percents', 'means_result', 'relevant_interactions_result', 'significant_means', 'interaction_scores'])

In [46]:
res['relevant_interactions_result']

,id_cp_interaction,interacting_pair,partner_a,partner_b,gene_a,gene_b,secreted,receptor_a,receptor_b,annotation_strategy,is_integrin,Corpus Spongiosum_male|Corpus Spongiosum_male,Corpus Spongiosum_male|Urethral Plate_male,Urethral Plate_male|Corpus Spongiosum_male,Urethral Plate_male|Urethral Plate_male,Corpus Spongiosum_female|Corpus Spongiosum_female,Corpus Spongiosum_female|Urethral Plate_female,Urethral Plate_female|Corpus Spongiosum_female,Urethral Plate_female|Urethral Plate_female
0,CPI-SC0A2DB962D,CDH1_integrin_a2b1_complex,simple:P12830,complex:integrin_a2b1_complex,CDH1,NaN,False,False,False,curated,True,0,0,0,0,0,0,0,1
2,CPI-SC0C8B7BCBB,COL11A1_integrin_a2b1_complex,simple:P12107,complex:integrin_a2b1_complex,COL11A1,NaN,True,False,False,curated,True,0,0,0,0,0,1,0,0
3,CPI-SC0D3C12C3F,COL11A2_integrin_a2b1_complex,simple:P13942,complex:integrin_a2b1_complex,COL11A2,NaN,True,False,False,curated,True,0,0,0,0,0,1,0,0
4,CPI-SC0B86B7CED,COL12A1_integrin_a2b1_complex,simple:Q99715,complex:integrin_a2b1_complex,COL12A1,NaN,True,False,False,curated,True,0,0,0,0,0,1,0,0
5,CPI-SC0FA343CEF,COL13A1_integrin_a2b1_complex,simple:Q5TAT6,complex:integrin_a2b1_complex,COL13A1,NaN,False,False,False,curated,True,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2769,CPI-SS07B6C7A75,WNT6_SFRP2,simple:Q9Y6F9,simple:Q96HF1,WNT6,SFRP2,True,False,False,curated,False,0,0,0,0,0,0,1,0
2771,CPI-SS08F783A87,WNT7B_SFRP2,simple:P56706,simple:Q96HF1,WNT7B,SFRP2,True,False,False,curated,False,0,0,0,0,0,0,1,0
2813,CPI-SS017D5AA63,WNT2_WIF1,simple:P09544,simple:Q9Y5W5,WNT2,WIF1,True,False,False,curated,False,1,0,0,0,0,0,1,0
2817,CPI-SS068E97BDA,WNT4_WIF1,simple:P56705,simple:Q9Y5W5,WNT4,WIF1,True,False,False,curated,False,0,0,0,0,0,0,1,0


In [47]:
res['relevant_interactions_result'].head(3)

,id_cp_interaction,interacting_pair,partner_a,partner_b,gene_a,gene_b,secreted,receptor_a,receptor_b,annotation_strategy,is_integrin,Corpus Spongiosum_male|Corpus Spongiosum_male,Corpus Spongiosum_male|Urethral Plate_male,Urethral Plate_male|Corpus Spongiosum_male,Urethral Plate_male|Urethral Plate_male,Corpus Spongiosum_female|Corpus Spongiosum_female,Corpus Spongiosum_female|Urethral Plate_female,Urethral Plate_female|Corpus Spongiosum_female,Urethral Plate_female|Urethral Plate_female
0,CPI-SC0A2DB962D,CDH1_integrin_a2b1_complex,simple:P12830,complex:integrin_a2b1_complex,CDH1,NaN,False,False,False,curated,True,0,0,0,0,0,0,0,1
2,CPI-SC0C8B7BCBB,COL11A1_integrin_a2b1_complex,simple:P12107,complex:integrin_a2b1_complex,COL11A1,NaN,True,False,False,curated,True,0,0,0,0,0,1,0,0
3,CPI-SC0D3C12C3F,COL11A2_integrin_a2b1_complex,simple:P13942,complex:integrin_a2b1_complex,COL11A2,NaN,True,False,False,curated,True,0,0,0,0,0,1,0,0


In [48]:
res['deconvoluted_result'].head(3)

,gene_name,uniprot,is_complex,protein_name,complex_name,id_cp_interaction,Corpus Spongiosum_female,Corpus Spongiosum_male,Urethral Plate_female,Urethral Plate_male
gene,,,,,,,,,,
UBASH3B,UBASH3B,Q8TF42,True,UBS3B_HUMAN,Dehydroepiandrosterone_bySTS,CPI-CS09B8977D7,0.004,0.018,0.036,0.052
UBASH3B,UBASH3B,Q8TF42,True,UBS3B_HUMAN,Dehydroepiandrosterone_bySTS,CPI-CS05760BB78,0.004,0.018,0.036,0.052
UBASH3B,UBASH3B,Q8TF42,True,UBS3B_HUMAN,Dehydroepiandrosterone_bySTS,CPI-CS0259A0EB4,0.004,0.018,0.036,0.052


In [49]:
res['means_result'].head(3)

,id_cp_interaction,interacting_pair,partner_a,partner_b,gene_a,gene_b,secreted,receptor_a,receptor_b,annotation_strategy,is_integrin,Corpus Spongiosum_male|Corpus Spongiosum_male,Corpus Spongiosum_male|Urethral Plate_male,Urethral Plate_male|Corpus Spongiosum_male,Urethral Plate_male|Urethral Plate_male,Corpus Spongiosum_female|Corpus Spongiosum_female,Corpus Spongiosum_female|Urethral Plate_female,Urethral Plate_female|Corpus Spongiosum_female,Urethral Plate_female|Urethral Plate_female
0,CPI-SC0A2DB962D,CDH1_integrin_a2b1_complex,simple:P12830,complex:integrin_a2b1_complex,CDH1,NaN,False,False,False,curated,True,0.019,0.045,0.671,0.697,0.000,0.000,0.516,0.647
2,CPI-SC0C8B7BCBB,COL11A1_integrin_a2b1_complex,simple:P12107,complex:integrin_a2b1_complex,COL11A1,NaN,True,False,False,curated,True,0.180,0.206,0.026,0.052,0.219,0.350,0.018,0.149
3,CPI-SC0D3C12C3F,COL11A2_integrin_a2b1_complex,simple:P13942,complex:integrin_a2b1_complex,COL11A2,NaN,True,False,False,curated,True,0.106,0.132,0.009,0.035,0.100,0.231,0.014,0.145


In [50]:
res['significant_means'].head(3)

,id_cp_interaction,interacting_pair,partner_a,partner_b,gene_a,gene_b,secreted,receptor_a,receptor_b,annotation_strategy,is_integrin,rank,Corpus Spongiosum_male|Corpus Spongiosum_male,Corpus Spongiosum_male|Urethral Plate_male,Urethral Plate_male|Corpus Spongiosum_male,Urethral Plate_male|Urethral Plate_male,Corpus Spongiosum_female|Corpus Spongiosum_female,Corpus Spongiosum_female|Urethral Plate_female,Urethral Plate_female|Corpus Spongiosum_female,Urethral Plate_female|Urethral Plate_female
0,CPI-SC0A2DB962D,CDH1_integrin_a2b1_complex,simple:P12830,complex:integrin_a2b1_complex,CDH1,NaN,False,False,False,curated,True,0.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.647
257,CPI-SC08C485EAE,FN1_integrin_a3b1_complex,simple:P02751,complex:integrin_a3b1_complex,FN1,NaN,True,False,False,curated,True,0.125,NaN,NaN,NaN,NaN,NaN,3.726,NaN,NaN
258,CPI-SC0E5A1BA8E,LAMA3_integrin_a3b1_complex,simple:Q16787,complex:integrin_a3b1_complex,LAMA3,NaN,True,False,False,curated,True,0.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.350


In [51]:
res['interaction_scores'].head(3)

,id_cp_interaction,interacting_pair,partner_a,partner_b,gene_a,gene_b,secreted,receptor_a,receptor_b,annotation_strategy,is_integrin,Corpus Spongiosum_male|Corpus Spongiosum_male,Corpus Spongiosum_male|Urethral Plate_male,Urethral Plate_male|Corpus Spongiosum_male,Urethral Plate_male|Urethral Plate_male,Corpus Spongiosum_female|Corpus Spongiosum_female,Corpus Spongiosum_female|Urethral Plate_female,Urethral Plate_female|Corpus Spongiosum_female,Urethral Plate_female|Urethral Plate_female
0,CPI-SC0A2DB962D,CDH1_integrin_a2b1_complex,simple:P12830,complex:integrin_a2b1_complex,CDH1,NaN,False,False,False,curated,True,0.0,0.0,0.0,0.0,0.0,0.000,0.0,76.547
2,CPI-SC0C8B7BCBB,COL11A1_integrin_a2b1_complex,simple:P12107,complex:integrin_a2b1_complex,COL11A1,NaN,True,False,False,curated,True,0.0,0.0,0.0,0.0,0.0,100.000,0.0,0.000
3,CPI-SC0D3C12C3F,COL11A2_integrin_a2b1_complex,simple:P13942,complex:integrin_a2b1_complex,COL11A2,NaN,True,False,False,curated,True,0.0,0.0,0.0,0.0,0.0,93.625,0.0,0.000


In [52]:
from cellphonedb.utils import search_utils

search_results = search_utils.search_analysis_results(
    query_cell_types_1 = ['Corpus Spongiosum_male', 'Urethral Plate_male'],  # List of cells 1, will be paired to cells 2 (list or 'All').
    query_cell_types_2 = ['Urethral Plate_male', 'Corpus Spongiosum_male'],     # List of cells 2, will be paired to cells 1 (list or 'All').
#     query_genes = ['LRP5'],                                       # filter interactions based on the genes participating (list).
#     query_interactions = ['CSF1_CSF1R'],                            # filter intereactions based on their name (list).
    significant_means = res['significant_means'],                          # significant_means file generated by CellPhoneDB.
    deconvoluted = res['deconvoluted_result'],                                    # devonvoluted file generated by CellPhoneDB.
    separator = '|',                                                # separator (default: |) employed to split cells (cellA|cellB).
    long_format = True                                              # converts the output into a wide table, removing non-significant interactions
)

# search_results.head(60)
search_results = search_results[search_results['interacting_cells'].isin([
                                                                         'Corpus Spongiosum_male|Urethral Plate_male', 
'Corpus Spongiosum_male|Corpus Spongiosum_male', 'Urethral Plate_male|Corpus Spongiosum_male', 'Urethral Plate_male|Corpus Spongiosum_male'])]
print(search_results.shape)

search_results.sort_values('significant_mean', ascending=False).head(100)

(105, 7)


,interacting_pair,partner_a,partner_b,gene_a,gene_b,interacting_cells,significant_mean
227,IGF2_IGF1R,simple:P01344,simple:P08069,IGF2,IGF1R,Urethral Plate_male|Corpus Spongiosum_male,19.709999
239,IGF2_IGF2R,simple:P01344,simple:P11717,IGF2,IGF2R,Urethral Plate_male|Corpus Spongiosum_male,19.697001
231,LGALS3_MERTK,simple:P17931,simple:Q12866,LGALS3,MERTK,Urethral Plate_male|Corpus Spongiosum_male,8.229000
274,IGFBP3_TMEM219,simple:P17936,simple:Q86XT9,IGFBP3,TMEM219,Urethral Plate_male|Corpus Spongiosum_male,5.145000
74,WNT2_SFRP2,simple:P09544,simple:Q96HF1,WNT2,SFRP2,Corpus Spongiosum_male|Corpus Spongiosum_male,3.153000
...,...,...,...,...,...,...,...
190,PLAUR_integrin_a4b1_complex,simple:Q03405,complex:integrin_a4b1_complex,PLAUR,NaN,Urethral Plate_male|Corpus Spongiosum_male,0.344000
153,atRetinoicAcid_byALDH1A3_RAreceptor_RXRB,complex:atRetinoicAcid_byALDH1A3,complex:RAreceptor_RXRB,NaN,NaN,Corpus Spongiosum_male|Urethral Plate_male,0.332000
136,atRetinoicAcid_byALDH1A3_RAreceptor_RARB_RXRB,complex:atRetinoicAcid_byALDH1A3,complex:RAreceptor_RARB_RXRB,NaN,NaN,Corpus Spongiosum_male|Urethral Plate_male,0.332000
162,atRetinoicAcid_byALDH1A3_RAreceptor_RARA_RXRA,complex:atRetinoicAcid_byALDH1A3,complex:RAreceptor_RARA_RXRA,NaN,NaN,Corpus Spongiosum_male|Urethral Plate_male,0.314000


In [55]:
search_results.to_csv('/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Genital_Tubercle_MPW/output/corpus_spongiosum_urethral_plate_signalling_males_MPW.csv')


In [54]:
from cellphonedb.utils import search_utils

search_results = search_utils.search_analysis_results(
    query_cell_types_1 = ['Corpus Spongiosum_male'],  # List of cells 1, will be paired to cells 2 (list or 'All').
    query_cell_types_2 = ['Urethral Plate_male'],     # List of cells 2, will be paired to cells 1 (list or 'All').
#     query_genes = ['LRP5'],                                       # filter interactions based on the genes participating (list).
#     query_interactions = ['CSF1_CSF1R'],                            # filter intereactions based on their name (list).
    significant_means = res['significant_means'],                          # significant_means file generated by CellPhoneDB.
    deconvoluted = res['deconvoluted_result'],                                    # devonvoluted file generated by CellPhoneDB.
    separator = '|',                                                # separator (default: |) employed to split cells (cellA|cellB).
    long_format = True                                              # converts the output into a wide table, removing non-significant interactions
)

# search_results.head(60)
search_results = search_results[search_results['interacting_cells'].isin([
                                                                         'Corpus Spongiosum_male|Urethral Plate_male'])]
print(search_results.shape)

search_results.sort_values('significant_mean', ascending=False).head(55)

(54, 7)


,interacting_pair,partner_a,partner_b,gene_a,gene_b,interacting_cells,significant_mean
62,DLK1_NOTCH2,simple:P80370,simple:Q04721,DLK1,NOTCH2,Corpus Spongiosum_male|Urethral Plate_male,2.886
61,DLK1_NOTCH3,simple:P80370,simple:Q9UM47,DLK1,NOTCH3,Corpus Spongiosum_male|Urethral Plate_male,2.854
37,JAG1_CD46,simple:P78504,simple:P15529,JAG1,CD46,Corpus Spongiosum_male|Urethral Plate_male,1.913
48,PRNP_ADGRG6,simple:F7VJQ1,simple:Q86SQ4,PRNP,ADGRG6,Corpus Spongiosum_male|Urethral Plate_male,1.203
12,COL4A2_ADGRG6,simple:P08572,simple:Q86SQ4,COL4A2,ADGRG6,Corpus Spongiosum_male|Urethral Plate_male,1.125
60,GJA1_GJA1,simple:P17302,simple:P17302,GJA1,GJA1,Corpus Spongiosum_male|Urethral Plate_male,1.092
6,LRRC4B_PTPRD,simple:Q9NT99,simple:P23468,LRRC4B,PTPRD,Corpus Spongiosum_male|Urethral Plate_male,1.080
7,LRFN5_PTPRD,simple:Q96NI6,simple:P23468,LRFN5,PTPRD,Corpus Spongiosum_male|Urethral Plate_male,1.054
8,NLGN3_PTPRD,simple:Q9NZ94,simple:P23468,NLGN3,PTPRD,Corpus Spongiosum_male|Urethral Plate_male,1.027
2,SEMA3A_NRP1,simple:Q14563,simple:O14786,SEMA3A,NRP1,Corpus Spongiosum_male|Urethral Plate_male,0.880


In [25]:
from cellphonedb.utils import search_utils

search_results = search_utils.search_analysis_results(
    query_cell_types_1 = ['Urethral Plate_male'],  # List of cells 1, will be paired to cells 2 (list or 'All').
    query_cell_types_2 = ['Corpus Spongiosum_male'],     # List of cells 2, will be paired to cells 1 (list or 'All').
#     query_genes = ['LRP5'],                                       # filter interactions based on the genes participating (list).
#     query_interactions = ['CSF1_CSF1R'],                            # filter intereactions based on their name (list).
    significant_means = res['significant_means'],                          # significant_means file generated by CellPhoneDB.
    deconvoluted = res['deconvoluted_result'],                                    # devonvoluted file generated by CellPhoneDB.
    separator = '|',                                                # separator (default: |) employed to split cells (cellA|cellB).
    long_format = True                                              # converts the output into a wide table, removing non-significant interactions
)

# search_results.head(60)
search_results = search_results[search_results['interacting_cells'].isin([
                                                                         'Urethral Plate_male|Corpus Spongiosum_male'])]
print(search_results.shape)

search_results.sort_values('significant_mean', ascending=False).head(50)

(27, 7)


,interacting_pair,partner_a,partner_b,gene_a,gene_b,interacting_cells,significant_mean
103,IGF2_IGF1R,simple:P01344,simple:P08069,IGF2,IGF1R,Urethral Plate_male|Corpus Spongiosum_male,19.709999
104,IGF2_IGF2R,simple:P01344,simple:P11717,IGF2,IGF2R,Urethral Plate_male|Corpus Spongiosum_male,19.697001
100,LGALS3_MERTK,simple:P17931,simple:Q12866,LGALS3,MERTK,Urethral Plate_male|Corpus Spongiosum_male,8.229000
105,IGFBP3_TMEM219,simple:P17936,simple:Q86XT9,IGFBP3,TMEM219,Urethral Plate_male|Corpus Spongiosum_male,5.145000
78,WNT5B_SFRP2,simple:Q9H1J7,simple:Q96HF1,WNT5B,SFRP2,Urethral Plate_male|Corpus Spongiosum_male,2.904000
75,WNT7B_SFRP2,simple:P56706,simple:Q96HF1,WNT7B,SFRP2,Urethral Plate_male|Corpus Spongiosum_male,2.903000
114,WNT5A_SFRP2,simple:P41221,simple:Q96HF1,WNT5A,SFRP2,Urethral Plate_male|Corpus Spongiosum_male,2.831000
110,WNT4_SFRP2,simple:P56705,simple:Q96HF1,WNT4,SFRP2,Urethral Plate_male|Corpus Spongiosum_male,2.824000
90,CD55_ADGRE5,simple:P08174,simple:P48960,CD55,ADGRE5,Urethral Plate_male|Corpus Spongiosum_male,2.005000
95,PDGFA_PDGFRA,simple:P04085,simple:P16234,PDGFA,PDGFRA,Urethral Plate_male|Corpus Spongiosum_male,1.929000
